# Entity Management with Nepal Entity Service

This notebook demonstrates how to manage entities using the Nepal Entity Service (nes). We'll work with authentic Nepali political data including politicians, political parties, and government bodies.

## Topics Covered

1. Initialize the Publication Service
2. Create new entities
3. Retrieve and display entities
4. Update entities with automatic versioning
5. View version history
6. Delete entities (if needed)

## Prerequisites

Make sure you have installed the nes package:
```bash
poetry install
```

## 1. Setup and Initialization

In [ ]:
# Import required modules
from pathlib import Path
from datetime import datetime, date

from nes.database.file_database import FileDatabase
from nes.services.publication import PublicationService
from nes.services.search import SearchService

In [ ]:
# Initialize database and services
db_path = Path("../nes-db/v2")
db = FileDatabase(base_path=str(db_path))

pub_service = PublicationService(database=db)
search_service = SearchService(database=db)

print("✓ Services initialized successfully")
print(f"  Database path: {db_path}")

## 2. Create a New Entity

Let's create a new politician entity with authentic Nepali data.

In [ ]:
# Define entity data for a Nepali politician
politician_data = {
    "slug": "gagan-thapa",
    "type": "person",
    "sub_type": "politician",
    "names": [
        {
            "kind": "PRIMARY",
            "en": {
                "full": "Gagan Kumar Thapa",
                "first": "Gagan Kumar",
                "last": "Thapa",
            },
            "ne": {"full": "गगन कुमार थापा", "first": "गगन कुमार", "last": "थापा"},
        },
        {"kind": "ALIAS", "en": {"full": "Gagan Thapa"}, "ne": {"full": "गगन थापा"}},
    ],
    "attributes": {
        "party": "nepali-congress",
        "constituency": "Kathmandu-4",
        "positions": ["Member of Parliament", "General Secretary"],
        "birth_year": "1976",
    },
}

print("Entity data prepared:")
print(f"  Name: {politician_data['names'][0]['en']['full']}")
print(f"  Nepali: {politician_data['names'][0]['ne']['full']}")
print(f"  Party: {politician_data['attributes']['party']}")

In [ ]:
# Create the entity
try:
    entity = await pub_service.create_entity(
        entity_data=politician_data,
        author_id="author:human:notebook-user",
        change_description="Initial creation via notebook",
    )

    print("✓ Entity created successfully!")
    print(f"  ID: {entity.id}")
    print(f"  Version: {entity.version_summary.version_number}")
    print(f"  Created at: {entity.version_summary.created_at}")

except ValueError as e:
    print(f"⚠ Entity may already exist: {e}")
    # Try to retrieve it instead
    from nes.core.identifiers import build_entity_id

    entity_id = build_entity_id("person", "politician", "gagan-thapa")
    entity = await pub_service.get_entity(entity_id)
    if entity:
        print(f"  Retrieved existing entity: {entity.id}")

## 3. Retrieve and Display Entities

Let's retrieve entities and display their information.

In [ ]:
# Retrieve a specific entity
entity_id = "entity:person/gagan-thapa"
entity = await pub_service.get_entity(entity_id)

if entity:
    print("Entity Details:")
    print("=" * 60)
    print(f"ID: {entity.id}")
    print(f"Type: {entity.type} / {entity.sub_type}")
    print(f"\nNames:")
    for name in entity.names:
        print(f"  {name.kind}:")
        print(f"    English: {name.en.full}")
        print(f"    Nepali: {name.ne.full}")

    print(f"\nAttributes:")
    for key, value in entity.attributes.items():
        print(f"  {key}: {value}")

    print(f"\nVersion Info:")
    print(f"  Version: {entity.version_summary.version_number}")
    print(f"  Created: {entity.version_summary.created_at}")
    print(f"  Author: {entity.version_summary.author.slug}")
else:
    print(f"Entity not found: {entity_id}")

In [ ]:
# Search for entities
results = await search_service.search_entities(
    query="thapa", entity_type="person", limit=5
)

print(f"Search Results for 'thapa': {len(results)} found")
print("=" * 60)

for entity in results:
    print(f"\n{entity.names[0].en.full}")
    print(f"  ID: {entity.id}")
    print(f"  Nepali: {entity.names[0].ne.full}")
    if entity.attributes and "party" in entity.attributes:
        print(f"  Party: {entity.attributes['party']}")

## 4. Update an Entity

Let's update the entity with new information. This will automatically create a new version.

In [ ]:
# Get the entity
entity = await pub_service.get_entity("entity:person/gagan-thapa")

if entity:
    print(f"Current version: {entity.version_summary.version_number}")
    print(f"Current attributes: {entity.attributes}")

    # Update attributes
    entity.attributes["positions"] = [
        "Member of Parliament",
        "General Secretary of Nepali Congress",
        "Minister of Health and Population",
    ]
    entity.attributes["education"] = "Tribhuvan University"

    # Update the entity
    updated_entity = await pub_service.update_entity(
        entity=entity,
        author_id="author:human:notebook-user",
        change_description="Added ministerial position and education info",
    )

    print(f"\n✓ Entity updated!")
    print(f"New version: {updated_entity.version_summary.version_number}")
    print(f"Updated attributes: {updated_entity.attributes}")
else:
    print("Entity not found")

## 5. View Version History

Let's explore the complete version history of the entity.

In [ ]:
# Get version history
entity_id = "entity:person/gagan-thapa"
versions = await pub_service.get_entity_versions(entity_id)

print(f"Version History for {entity_id}")
print("=" * 60)
print(f"Total versions: {len(versions)}\n")

for version in versions:
    print(f"Version {version.version_number}:")
    print(f"  Created: {version.created_at}")
    print(f"  Author: {version.author.slug}")
    print(f"  Description: {version.change_description or '(no description)'}")

    # Show snapshot summary
    if version.snapshot and "attributes" in version.snapshot:
        attrs = version.snapshot["attributes"]
        print(f"  Attributes: {list(attrs.keys())}")

    print()

In [ ]:
# Compare two versions
if len(versions) >= 2:
    v1 = versions[0]
    v2 = versions[1]

    print(f"Comparing Version {v1.version_number} vs Version {v2.version_number}")
    print("=" * 60)

    attrs_v1 = v1.snapshot.get("attributes", {})
    attrs_v2 = v2.snapshot.get("attributes", {})

    # Find added attributes
    added = {k: v for k, v in attrs_v2.items() if k not in attrs_v1}
    if added:
        print("\nAdded:")
        for key, value in added.items():
            print(f"  + {key}: {value}")

    # Find modified attributes
    modified = {
        k: (attrs_v1[k], attrs_v2[k])
        for k in attrs_v2
        if k in attrs_v1 and attrs_v1[k] != attrs_v2[k]
    }
    if modified:
        print("\nModified:")
        for key, (old, new) in modified.items():
            print(f"  ~ {key}:")
            print(f"      Old: {old}")
            print(f"      New: {new}")

    # Find removed attributes
    removed = {k: v for k, v in attrs_v1.items() if k not in attrs_v2}
    if removed:
        print("\nRemoved:")
        for key, value in removed.items():
            print(f"  - {key}: {value}")

## 6. Create Multiple Entities (Batch)

Let's create multiple entities at once.

In [ ]:
# Define multiple politicians
politicians = [
    {
        "slug": "bimalendra-nidhi",
        "type": "person",
        "sub_type": "politician",
        "names": [
            {
                "kind": "PRIMARY",
                "en": {
                    "full": "Bimalendra Nidhi",
                    "first": "Bimalendra",
                    "last": "Nidhi",
                },
                "ne": {"full": "विमलेन्द्र निधि", "first": "विमलेन्द्र", "last": "निधि"},
            }
        ],
        "attributes": {
            "party": "nepali-congress",
            "constituency": "Dhanusha-3",
            "positions": ["Deputy Prime Minister", "Minister"],
        },
    },
    {
        "slug": "rabi-lamichhane",
        "type": "person",
        "sub_type": "politician",
        "names": [
            {
                "kind": "PRIMARY",
                "en": {
                    "full": "Rabi Lamichhane",
                    "first": "Rabi",
                    "last": "Lamichhane",
                },
                "ne": {"full": "रवि लामिछाने", "first": "रवि", "last": "लामिछाने"},
            }
        ],
        "attributes": {
            "party": "rastriya-swatantra-party",
            "constituency": "Chitwan-2",
            "positions": ["Home Minister", "Party President"],
        },
    },
]

# Create entities
created_count = 0
skipped_count = 0

for politician_data in politicians:
    try:
        entity = await pub_service.create_entity(
            entity_data=politician_data,
            author_id="author:human:notebook-user",
            change_description="Batch creation via notebook",
        )
        print(f"✓ Created: {entity.names[0].en.full}")
        created_count += 1
    except ValueError as e:
        print(
            f"⚠ Skipped: {politician_data['names'][0]['en']['full']} (already exists)"
        )
        skipped_count += 1

print(f"\nSummary: {created_count} created, {skipped_count} skipped")

## 7. Summary and Next Steps

In this notebook, we've learned how to:

- ✓ Initialize the Publication Service
- ✓ Create new entities with multilingual names
- ✓ Retrieve and search for entities
- ✓ Update entities with automatic versioning
- ✓ View and compare version history
- ✓ Create multiple entities in batch

### Next Steps

1. Explore **Relationship Management** in `02_relationship_management.ipynb`
2. Learn about **Data Import Workflows** in `03_data_import_workflow.ipynb`
3. Perform **Data Quality Analysis** in `04_data_quality_analysis.ipynb`

### Resources

- Data Maintainer Guide: `docs/data-maintainer-guide.md`
- Example Scripts: `examples/`
- API Documentation: Run the server and visit `/docs`